In [174]:
# === 0) Imports necesarios ===
import pandas as pd
import numpy as np
from prophet import Prophet


In [175]:
# === 1) Leer y preparar los datos para el modelo, variable X y variable Y ===

df = pd.read_csv("ventas_mensuales - TODAS-Mod.csv", sep=";", decimal=".", encoding="utf-8") #Leer el archicvo CSV con separador ; y codificación Latin1
df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y") #Convertir la columna 'date' a formato datetime
ventas_col = "value" # Nombre de la columna de ventas
familia_col = "Familia" # Nombre de la columna de familia
y = df.set_index("date").sort_index()["value"]

In [176]:
# === 2) Crear tabla pivot por familias y calcular crecimiento porcentual ===

# Crear tabla pivot con fechas como índice, familias como columnas y suma de ventas como valores
df_pivot = df.pivot_table(
    index="date",
    columns=familia_col,
    values=ventas_col,
    aggfunc="sum"
).sort_index()


In [177]:
# === 3) Condicional para modelar según si se tiene encuenta el COVID o no ===

covid = 0 # 0 = Sin datos de la etapa COVID; 1 = Con datos de la etapa COVID
COVID_START = "2020-03-01" # Fecha considerada de inicio del periodo COVID
COVID_END   = "2021-12-01" # Fecha considerada de fin del periodo COVID

# Condicional para enmascarar los datos del periodo COVID si covid == 0
if covid == 0:
    y_masked = y.copy()
    y_masked.loc[COVID_START:COVID_END] = np.nan
    
    df_pivot_masked = df_pivot.copy()
    df_pivot_masked.loc[COVID_START:COVID_END, :] = np.nan

else:  
    y_masked = y.copy()
    df_pivot_masked = df_pivot.copy()

pivot_used = df_pivot_masked if covid == 0 else df_pivot

In [178]:
# === 3b) Transformación logarítmica de los datos para modelar según si se tiene en cuenta el COVID o no ===

y_masked_log   = np.log1p(y_masked) # Transformación logarítmica de la variable Y
pivot_used_log = np.log1p(pivot_used) # Transformación logarítmica de las variables X

In [179]:
# === 4) MATRIZ DE CORRELACIÓN ENTRE FAMILIAS (CRECIMIENTOS) ===

crec_familias = df_pivot.pct_change() # Calcular crecimiento porcentual mensual entre familias
corr_familias = crec_familias.corr() # Calcular matriz de correlación entre familias basándose en los crecimientos porcentuales
display(corr_familias) # Mostrar la matriz de correlación

Familia,Mesa,Mini Tumbona,Parasol,Sillas,Tumbona
Familia,,,,,
Mesa,1.000000,-0.074710,0.234815,-0.135872,-0.260789
Mini Tumbona,-0.074710,1.000000,-0.034962,0.275357,0.037756
Parasol,0.234815,-0.034962,1.000000,-0.016748,-0.033249
Sillas,-0.135872,0.275357,-0.016748,1.000000,0.245350
Tumbona,-0.260789,0.037756,-0.033249,0.245350,1.000000


In [180]:
# === 5) PROBABILIDAD P(fam2 sube | fam1 sube) ===

n_familias = len(df_pivot.columns) # Número de familias en el DataFrame pivot

# Condicional para calcular la probabilidad solo si hay al menos 2 familias
if n_familias >= 2:
    prob_subida_dict = {}
    
    # Calcular P(fam2 sube | fam1 sube) para cada par de familias
    for fam1 in df_pivot.columns:
        for fam2 in df_pivot.columns:
            if fam1 == fam2:
                continue

            A = crec_familias[fam1] > 0   # fam1 sube
            B = crec_familias[fam2] > 0   # fam2 sube

            # Calcular la probabilidad condicional P(fam2 sube | fam1 sube)
            if A.sum() > 0:
                prob = (A & B).sum() / A.sum()
                prob_subida_dict[(fam1, fam2)] = prob

    prob_subida = pd.Series(prob_subida_dict).unstack() # Convertir el diccionario a DataFrame para mejor visualización
    print("\nPROBABILIDAD: P(fam2 sube | fam1 sube)")
    display(prob_subida)
else:
    print("\nSólo hay una familia, no se puede calcular P(fam2 sube | fam1 sube).")
    prob_subida = pd.DataFrame()


PROBABILIDAD: P(fam2 sube | fam1 sube)


,Mesa,Mini Tumbona,Parasol,Sillas,Tumbona
Mesa,NaN,0.45614,0.631579,0.596491,0.473684
Mini Tumbona,0.448276,NaN,0.689655,0.517241,0.603448
Parasol,0.521739,0.57971,NaN,0.579710,0.594203
Sillas,0.531250,0.46875,0.625000,NaN,0.593750
Tumbona,0.457627,0.59322,0.694915,0.644068,NaN


In [181]:
# === 6) SELECCIÓN DE FAMILIAS RELACIONADAS SEGÚN PROB_SUBIDA ===

umbral_prob = 0.6 # Umbral de probabilidad para considerar una relación fuerte
relaciones_fuertes = {}   # Diccionario para almacenar relaciones fuertes

# Extraer relaciones fuertes basadas en el umbral de probabilidad
if not prob_subida.empty:
    # Iterar sobre cada familia y sus probabilidades condicionales
    for fam1 in prob_subida.index:
        rels = prob_subida.loc[fam1].dropna()
        fuertes = rels[rels >= umbral_prob].index.tolist()
        # Si hay relaciones fuertes, añadirlas al diccionario
        if fuertes:
            relaciones_fuertes[fam1] = fuertes

# Mostrar las relaciones fuertes encontradas
print(f"\nRelaciones fuertes (P>= {umbral_prob:.2f}):")
for k, v in relaciones_fuertes.items():
    print(f"{k} → {v}")
else:
    if prob_subida.empty:
        print("\nNo hay matriz de prob_subida (sólo una familia o datos insuficientes).")


Relaciones fuertes (P>= 0.60):
Mesa → ['Parasol']
Mini Tumbona → ['Parasol', 'Tumbona']
Sillas → ['Parasol']
Tumbona → ['Parasol', 'Sillas']


In [182]:
# === 7) Preparar las variables exógenas ===

cols_exog = ["Sol","Precipitación","Agosto","Tendencia","Festivos","Temperatura","ICC"]  # Con todas las variables exógenas disponibles.
cols_exog = [c for c in cols_exog if c in df.columns] # Filtrar solo las columnas que existen en el DataFrame

for c in cols_exog: # Transformar y preparar las variables exógenas
    df[c] = df[c].astype(str) # Convertir a string para reemplazar comas
    df[c] = df[c].str.replace(",", ".", regex=False) # Reemplazar comas por puntos decimales
    df[c] = pd.to_numeric(df[c], errors="coerce") # Convertir a numérico, forzando errores a NaN

# Verificar tipos de datos de las columnas exógenas
# print("Columnas del df:")
# print(df.columns)
# df[cols_exog].dtypes

# Preparar la variable X con las columnas exógenas
X = (
    df[["date"] + cols_exog]
    .drop_duplicates(subset="date")
    .set_index("date")
    .sort_index()
)

In [183]:
# === 8) Preparar DataFrame final para Prophet con variable Y y variables exógenas + familias relacionadas ===

FAM_OBJ = "Tumbona" # Familia objetivo

# Comprobar que la familia objetivo existe en el DataFrame pivot
if FAM_OBJ not in pivot_used_log.columns:
    raise ValueError(f"La familia objetivo {FAM_OBJ} no existe en tabla_familias.columns")

familias_rel = relaciones_fuertes.get(FAM_OBJ, []) # Familias relacionadas según prob_subida
print(f"Familias relacionadas con {FAM_OBJ} (P>= {umbral_prob:.2f}):", familias_rel) # Familias relacionadas según prob_subida

LAG_FAM = 1 # Número de periodos de lag para las familias relacionadas



df_prophet = pd.DataFrame(index=pivot_used_log.index) # DataFrame final para Prophet
df_prophet["ds"] = df_prophet.index # Fecha como índice
df_prophet["y"]  = pivot_used_log[FAM_OBJ]  # Variable Y para la familia objetivo


df_prophet = df_prophet.join(X, how="left") # añadir variables exógenas

# Añadir variables de familias relacionadas con lag
for fam_rel in familias_rel:
    col_name = f"{fam_rel}_lag{LAG_FAM}"
    df_prophet[col_name] = pivot_used_log[fam_rel].shift(LAG_FAM)

df_prophet = df_prophet.dropna() # Eliminar filas con valores NaN
df_prophet = df_prophet.sort_values("ds") # Ordenar por fecha

# # Guardar el DataFrame final para Prophet en un archivo Excel
# df_prophet.to_excel("df_prophet_final.xlsx")
print(df_prophet.head(10))


Familias relacionadas con Tumbona (P>= 0.60): ['Parasol', 'Sillas']
                   ds         y   Sol  Precipitación  Agosto  Tendencia  \
date                                                                      
2015-02-01 2015-02-01  8.696343  3.74           0.36       0         22   
2015-03-01 2015-03-01  9.490167  5.74           1.91       0         51   
2015-04-01 2015-04-01  9.641538  8.04           0.37       0         58   
2015-05-01 2015-05-01  9.445966  9.14           0.93       0         59   
2015-06-01 2015-06-01  9.035392  9.77           0.36       0         53   
2015-07-01 2015-07-01  9.445412  8.38           0.45       0         47   
2015-08-01 2015-08-01  8.001690  8.78           1.36       1         34   
2015-09-01 2015-09-01  8.114025  5.96           1.37       0         20   
2015-10-01 2015-10-01  8.471568  3.86           2.41       0         14   
2015-11-01 2015-11-01  8.554682  3.58           0.60       0         11   

            Festivos  Temperatu

In [184]:
# === 9) Definición de funciones para el cálculo de métricas de evaluación ===

# Root Mean Squared Error (RMSE)
def rmse(y_verd, y_pred):
    return np.sqrt(np.mean((y_pred - y_verd)**2))

# Mean Absolute Error (MAE) 
def mae(y_verd, y_pred):
    return np.mean(np.abs(y_pred - y_verd))

# Mean Absolute Percentage Error (MAPE) con denominador seguro
def mape_safe(y_verd, y_pred, eps=1.0):
    denom = np.maximum(np.abs(y_verd), eps)
    return np.mean(np.abs(y_pred - y_verd) / denom) * 100.0

# Symmetric Mean Absolute Percentage Error (SMAPE)
def smape(y_verd, y_pred, eps=1e-8):
    denom = np.abs(y_verd) + np.abs(y_pred)
    denom = np.where(denom < eps, eps, denom)
    return np.mean(2.0 * np.abs(y_pred - y_verd) / denom) * 100.0

# Weighted Absolute Percentage Error (WAPE)
def wape(y_verd, y_pred, eps=1e-8):
    return np.sum(np.abs(y_pred - y_verd)) / max(np.sum(np.abs(y_verd)), eps) * 100.0

# Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y_verd, y_pred):
    yt = np.log1p(np.maximum(y_verd, 0))
    yp = np.log1p(np.maximum(y_pred, 0))
    return np.sqrt(np.mean((yp - yt)**2))

# R-squared (R2) Score
def r2_score(y_verd, y_pred):
    var = np.var(y_verd)
    if var <= 0:
        return np.nan
    return 1.0 - np.sum((y_pred - y_verd)**2) / np.sum((y_verd - np.mean(y_verd))**2)

# Función para calcular todas las métricas anteriores
def metricas (y_verd, y_pred, eps_mape=1.0, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_verd, y_pred),
        f"{prefix}RMSLE": rmsle(y_verd, y_pred),
        f"{prefix}RMSE": rmse(y_verd, y_pred),
        f"{prefix}MAE": mae(y_verd, y_pred),
        f"{prefix}MAPE_safe(%)": mape_safe(y_verd, y_pred, eps=eps_mape),
        f"{prefix}SMAPE(%)": smape(y_verd, y_pred),
        f"{prefix}WAPE(%)": wape(y_verd, y_pred),
    }


In [185]:
# === 10) Valores constantes de los hiperparmámetros ===

CPS = 0.5         # changepoint_prior_scale
SPS= 10   # seasonality_prior_scale constante en el modelo final         
YEARLY_ORDER=10 # Fourier para anual
REGRESSOR_PRIOR = 3 # prior scale constante para regresores exógenos en el modelo final        
PRIORS_BASE = {"Precipitación":10, "Agosto":10, "Sol":10, "Tendencia":10} # Priors base para cada variable exógena


In [186]:
# === 11) División del DataFrame final en conjuntos de entrenamiento y prueba ===

df_full = df_prophet.copy() # DataFrame completo para modelar

split = int(len(df_full) * 0.80) # Índice de división
train = df_full.iloc[:split].copy() # Conjunto de entrenamiento
test  = df_full.iloc[split:].copy() # Conjunto de prueba
# print(f"\nTamaño train: {len(train)}, tamaño test: {len(test)}")

In [187]:
# === 12) Definición y ajuste del modelo Prophet con variables exógenas y familias relacionadas ===

# Definir y ajustar el modelo Prophet con variables exógenas y familias relacionadas
m = Prophet(
       
        seasonality_mode='additive', # Tipo de estacionalidad
        changepoint_prior_scale=CPS, # Escala de prior para puntos de cambio
        seasonality_prior_scale=SPS, # Escala de prior para estacionalidad
        interval_width=0.95 # Intervalo de confianza del 95%

    )

m.add_seasonality(name='anual', period=365.25, fourier_order=YEARLY_ORDER) # Añadir estacionalidad anual con Fourier

# Bucle para añadir regresores exógenos al modelo
for c in cols_exog:
    # Condicional para añadir solo si la columna existe en el DataFrame
    if c in df_prophet.columns:
        m.add_regressor(c, prior_scale=PRIORS_BASE.get(c, 0.8), standardize=True)

# Bucle para añadir regresores de familias relacionadas con lag
for fam_rel in familias_rel:
    col_name = f"{fam_rel}_lag{LAG_FAM}"
    # Condicional para añadir solo si la columna existe en el DataFrame
    if col_name in df_prophet.columns:
        m.add_regressor(col_name)

m.fit(train) # Ajustar el modelo con el conjunto de entrenamiento

22:55:22 - cmdstanpy - INFO - Chain [1] start processing
22:55:22 - cmdstanpy - INFO - Chain [1] done processing


In [188]:
# === 13) Realizar predicciones y preparar DataFrame de resultados ===

future = df_prophet[["ds"] + cols_exog + [c for c in df_prophet.columns if c.endswith(f"_lag{LAG_FAM}")]].copy() # DataFrame futuro para predicciones

forecast = m.predict(future) # Realizar predicciones

forecast["yhat_real"]        = np.expm1(forecast["yhat"]) # Convertir predicciones logarítmicas a escala real
forecast["yhat_lower_real"]  = np.expm1(forecast["yhat_lower"]) # Convertir límite inferior a escala real
forecast["yhat_upper_real"]  = np.expm1(forecast["yhat_upper"]) # Convertir límite superior a escala real

# Creamos el DataFrame final de forecast alineado con df_prophet
df_forecast = df_prophet[["ds", "y"]].merge(
    forecast[["ds", "yhat", "yhat_lower", "yhat_upper", "yhat_real", "yhat_lower_real", "yhat_upper_real"]],
    on="ds",
    how="left"
)

df_forecast = df_forecast.set_index("ds").sort_index() # Establecer 'ds' como índice y ordenar
df_forecast["y_real"] = np.expm1(df_forecast["y"])

df_forecast_train = df_forecast.iloc[:split].copy() # Conjunto de forecast para entrenamiento
df_forecast_test = df_forecast.iloc[split:].copy() # Conjunto de forecast para prueba

# print("\nForecast Prophet (primeras filas):")
# display(df_forecast.head())

In [189]:
# === 14) Cálculo y visualización de métricas de evaluación del modelo ===

# Calcular métricas de evaluación para el conjunto de prueba
metrics = metricas(
    y_verd=df_forecast_test["y_real"],       
    y_pred=df_forecast_test["yhat_real"],  
    prefix="Prophet_"
)

# Visualizar las métricas calculadas
for k, v in metrics.items():
    suf = "%" if any(x in k for x in ["WAPE","SMAPE","MAPE"]) else ""
    print(f"{k}: {v:,.2f}{suf}")
    

Prophet_R2: 0.01
Prophet_RMSLE: 0.62
Prophet_RMSE: 2,585.90
Prophet_MAE: 2,087.54
Prophet_MAPE_safe(%): 46.89%
Prophet_SMAPE(%): 48.42%
Prophet_WAPE(%): 39.75%
